In [1]:
# --- SETUP ---
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')


# Cargar archivo .env
# Ruta relativa desde el notebook hasta el .env
dotenv_path = '../generadores/.env'  #

# Cargar el archivo .env desde la ruta especificada
load_dotenv(dotenv_path=dotenv_path)

# Leer variables de entorno
username = os.getenv("USUARIO_DB")
password = os.getenv("CLAVE_BD")

# Validar que se cargaron bien
print("Usuario:", username)
print("Contraseña cargada:", bool(password))  # No imprime la clave, pero verifica si existe

if not username or not password:
    raise ValueError("Las variables de entorno USUARIO_DB o CLAVE_BD no están definidas")

# Cadena de conexión
conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=upgradeserver-vf.database.windows.net;'
    'DATABASE=Banco;'
    f'UID={username};'
    f'PWD={password}'
)

# Conexión
conn = pyodbc.connect(conn_str)



ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# Unión de tablas
usuarios_empleo = usuarios.merge(empleo[['usuario_id', 'ingresos', 'tipo_empleo']], on='usuario_id', how='left')
usuarios_creditos = creditos.groupby('usuario_id').size().reset_index(name='cantidad_creditos')
usuarios_df = usuarios_empleo.merge(usuarios_creditos, on='usuario_id', how='left')
usuarios_df = usuarios_df.merge(activos_financieros[['usuario_id', 'valor_activos']], on='usuario_id', how='left')


In [ ]:
# Limpieza
usuarios_df['cantidad_creditos'] = usuarios_df['cantidad_creditos'].fillna(0)
usuarios_df['valor_activos'] = usuarios_df['valor_activos'].fillna(0)
usuarios_df[['provincia', 'estado_civil', 'tipo_empleo', 'nacionalidad']] = usuarios_df[
    ['provincia', 'estado_civil', 'tipo_empleo', 'nacionalidad']].fillna("Desconocido")

In [ ]:
# --- ENCODING & SCALING ---
cat_vars = ['provincia', 'estado_civil', 'tipo_empleo', 'nacionalidad']
for col in cat_vars:
    le = LabelEncoder()
    usuarios_df[col] = le.fit_transform(usuarios_df[col])

features = ['edad', 'ingresos', 'provincia', 'estado_civil', 'tipo_empleo', 'cantidad_creditos', 'valor_activos']
X_scaled = StandardScaler().fit_transform(usuarios_df[features])

In [ ]:
# --- CLUSTERING DE RIESGO ---
kmeans = KMeans(n_clusters=4, random_state=42)
usuarios_df['cluster_riesgo'] = kmeans.fit_predict(X_scaled)

sns.countplot(x='cluster_riesgo', data=usuarios_df, palette='tab10')
plt.title("Distribución de Usuarios por Clúster de Riesgo")
plt.show()

In [ ]:
# --- PREDICCION DE MOROSIDAD ---
X = usuarios_df[features]
y = usuarios_df['moroso']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d')
plt.title("Matriz de Confusión")
plt.show()

In [ ]:
# --- VALIDACION CRUZADA ---
scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
print(f"Accuracy promedio (CV): {scores.mean():.4f}")

In [ ]:
# --- GRIDSEARCHCV ---
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}
grid = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, scoring='roc_auc', n_jobs=-1)
grid.fit(X_train, y_train)

print("Mejores parámetros:", grid.best_params_)
best_model = grid.best_estimator_
y_pred_best = best_model.predict(X_test)

In [ ]:
# --- CURVA ROC ---
y_scores = best_model.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.2f})', color='darkorange')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('Falso Positivo')
plt.ylabel('Verdadero Positivo')
plt.title('Curva ROC')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# --- IMPORTANCIA DE VARIABLES ---
importances = pd.Series(best_model.feature_importances_, index=features).sort_values(ascending=False)
importances.plot(kind='barh', title='Importancia de Variables')
plt.show()